In [25]:
import math
import numpy as np
from math import gcd
from fractions import Fraction
from IPython.display import display, Math

def computeContinuedFraction(b, a):
    '''Computes the continued fraction decomposiiton of b/a'''
    arrDecomposition = []
    completed = False
    frac =  Fraction(b, a)
    while(completed==False):
        if(frac.denominator == 1):
            completed == True
            arrDecomposition.append(frac.numerator)
            break
        else:
            n = math.ceil(frac)
            arrDecomposition.append(n)
            newFraction = Fraction(int(n*frac.denominator)-frac.numerator, frac.denominator)
            invertedFraction = Fraction(newFraction.denominator, newFraction.numerator)
            frac = invertedFraction
    return(arrDecomposition)

def find_Seifert_invariants(b_1, b_2, b_3):
    invariants_found=False
    b=-1
    mult = 1
    if (b_1 > 100 or b_2 > 100 or b_3 > 100):
        mult=int(min(b_1, b_2, b_3)/10)
    while(invariants_found==False):
        for a_2 in range(1,mult*100,1):
            for a_3 in range(1,mult*100,1):
                for a_1 in range(1,mult*100,1):
                    if b_1*b_2*b_3*b + b_2*b_3*a_1 + b_1*b_3*a_2 + b_1*b_2*a_3==-1:
                        invariants_found==True
                        #print("The Seifert invariants are: " + str((b, a_1, a_2, a_3)))
                        return((b, a_1, a_2, a_3))
        b-=1

def linkingMatrix(b_1, b_2, b_3):
    (b, a_1, a_2, a_3) = find_Seifert_invariants(b_1, b_2, b_3)
    #print((b, a_1, a_2, a_3))
    leg_1_decomposition = -1*np.array(computeContinuedFraction(b_1, a_1))
    leg_2_decomposition = -1*np.array(computeContinuedFraction(b_2, a_2))
    leg_3_decomposition = -1*np.array(computeContinuedFraction(b_3, a_3))
    #print(leg_1_decomposition)
    #print(leg_2_decomposition)
    #print(leg_3_decomposition)
    s= 1 + leg_1_decomposition.shape[0] + leg_2_decomposition.shape[0] + leg_3_decomposition.shape[0]
    M = np.zeros((s,s))
    l_1 =  leg_1_decomposition.shape[0]
    l_2 = leg_1_decomposition.shape[0] + leg_2_decomposition.shape[0]
    l_3= s-1
    #print(l_1, l_2, l_3)
    M[0,0] = b
    M[0,1] = 1 # first vertex of first leg
    M[0,l_1+1] = 1 # first vertex of second leg
    M[0, l_2+1] = 1 # first vertex of third leg
    M[1,0] = 1 # first vertex of first leg
    M[l_1+1,0] = 1 # first vertex of second leg
    M[l_2+1,0] = 1 # first vertex of third leg
    '''Now we put in the continued fration decomposition on the diagonal for the first leg'''
    for i in range(1, l_1+1,1):
        M[i, i]=leg_1_decomposition[i-1]
        M[i, i-1] = 1
        if (i+1 != l_1+1):
            M[i, i+1] = 1         
        # The last if statement is to ensure that the terminal vertices only have degree 1
    '''Now we put in the continued fration decomposition on the diagonal for the second leg'''
    for i in range(l_1+1, l_2+1,1):
        M[i, i]=leg_2_decomposition[i-l_1-1]
        if (i+1 != l_2+1):
            M[i, i+1] = 1
        if (i-1 != l_1):
            M[i, i-1] = 1
        # The last two if statements are to ensure that the terminal vertices only have degree 1 
    '''Now we put in the continued fration decomposition on the diagonal for the third leg'''
    for i in range(l_2+1, s,1):
        #print(M)
        #print(i-l_2-1)
        M[i, i]=leg_3_decomposition[i-l_2-1]
        if (i+1 !=s):
            M[i, i+1] = 1
        if (i-1 != l_2):
            M[i, i-1] = 1
        # The last two if statements are to ensure that the terminal vertices only have degree 1 
    # changing 1's back to zeros
    '''M[p+2, p+1] = 0
    M[p+1, p+2] = 0  ''' 
    #print(M.shape) 
    return M

#linkingMatrix(2,5,7)

def sum_h(leg_1_length, leg_2_length, leg_3_length, M):
    s= 1 + leg_1_length + leg_2_length + leg_3_length
    l_1 = leg_1_length
    l_2 = leg_1_length + leg_2_length 
    h_1 = abs(round(np.linalg.det(np.delete(np.delete(M, leg_1_length, 0), leg_1_length,1))))
    h_2 = abs(round(np.linalg.det(np.delete(np.delete(M, l_2, 0), l_2,1))))
    h_3 = abs(round(np.linalg.det(np.delete(np.delete(M, s-1, 0), s-1,1))))

    return(round(h_1+h_2+h_3))

#computeContinuedFraction(7,5)

def Delta(b_1, b_2, b_3, leg_1_length, leg_2_length, leg_3_length, M, s):
    sum_m_v = int(np.trace(M))
    Delta= Fraction(sum_h(leg_1_length, leg_2_length, leg_3_length, M)-3*s - sum_m_v - Fraction(b_2*b_3,b_1) - Fraction(b_1*b_3,b_2) - Fraction(b_1*b_2,b_3),4)
    #print("Delta is: " + str(Delta))
    return Delta


def compute_Z_0(b_1, b_2, b_3, max_m):
    if(gcd(b_1, b_2)==1 and gcd(b_2, b_3)==1 and gcd(b_1, b_3)==1):
        v = b_1*b_2*b_3
        alpha_1 =  b_1*b_2*b_3 - b_1*b_2 -b_1*b_3 - b_2*b_3
        alpha_2 =  b_1 * b_2 * b_3 + b_1*b_2 -b_1*b_3 - b_2*b_3
        alpha_3 =  b_1 * b_2 * b_3 - b_1*b_2 +b_1*b_3 - b_2*b_3
        alpha_4 =  b_1 * b_2 * b_3 + b_1*b_2 +b_1*b_3 - b_2*b_3
        gamma = min(alpha_1, alpha_2, alpha_3, alpha_4)
        (b, a_1, a_2, a_3) = find_Seifert_invariants(b_1, b_2, b_3)
        leg_1_decomposition = -1*np.array(computeContinuedFraction(b_1, a_1))
        leg_2_decomposition = -1*np.array(computeContinuedFraction(b_2, a_2))
        leg_3_decomposition = -1*np.array(computeContinuedFraction(b_3, a_3))
        leg_1_length = leg_1_decomposition.shape[0]
        leg_2_length = leg_2_decomposition.shape[0]
        leg_3_length = leg_3_decomposition.shape[0]
        s= 1 + leg_1_decomposition.shape[0] + leg_2_decomposition.shape[0] + leg_3_decomposition.shape[0]
        M = linkingMatrix(b_1, b_2, b_3)
        delta_exponent = Delta(b_1, b_2, b_3, leg_1_length, leg_2_length, leg_3_length, M,s)+Fraction(gamma**2, 4*v)
        output = "\widehat{{Z}}_0(\Sigma({0}, {1}, {2});q) = q^{{{3}}}\\cdot\\left(".format(b_1, b_2, b_3,delta_exponent)
        for i in range(max_m):
            for x in [alpha_1, alpha_2, alpha_3, alpha_4]:        
                exponent_1 = Fraction(x**2-gamma**2,4*v) + x*i + i**2*v
                exponent_2 = Fraction(x**2-gamma**2,4*v) - x*i + i**2*v
                if x ==alpha_1 or x == alpha_4:
                    if i ==0:
                        if x ==gamma:
                            output+="+1"
                        else:
                            output+="+q^{{{0}}}".format(exponent_1)
                    else:
                        output+="+q^{{{0}}}-q^{{{1}}}".format(exponent_1, exponent_2)
                else:
                    if i ==0:
                        if x ==gamma:
                            output+="-1"
                        else:
                            output+="-q^{{{0}}}".format(exponent_1)
                    else:
                        output+="-q^{{{0}}}+q^{{{1}}}".format(exponent_1, exponent_2)
        output+="\\right)"
        #if M.shape[0] >200:
        #print(M.shape)
        #print(output)
        #display(Math(output))
        return delta_exponent
    else:
        print(str((b_1, b_2, b_3)) + " are not relatively coprime.")

compute_Z_0(3, 11, 25, 5)

Fraction(121, 2)

We compute for some b_1, b_2, b_3

In [26]:
'''import random

i= 0
while i <10:
    b_1 = random.randint(3,50)
    b_2 = random.randint(b_1,70)
    b_3 = random.randint(b_2, 100)
    if b_1 < b_2 and b_2<b_3 and (gcd(b_1, b_2)==1 and gcd(b_2, b_3)==1 and gcd(b_1, b_3)==1):
        s=compute_Z_0(b_1, b_2, b_3, 2)
        if s[0] > 200:
            print(s)
            i+=1'''

'import random\n\ni= 0\nwhile i <10:\n    b_1 = random.randint(3,50)\n    b_2 = random.randint(b_1,70)\n    b_3 = random.randint(b_2, 100)\n    if b_1 < b_2 and b_2<b_3 and (gcd(b_1, b_2)==1 and gcd(b_2, b_3)==1 and gcd(b_1, b_3)==1):\n        s=compute_Z_0(b_1, b_2, b_3, 2)\n        if s[0] > 200:\n            print(s)\n            i+=1'

Checking that $\Delta_0 = \frac{1}{2} + n$

In [27]:
counter = 0
for b_1 in range(2, 60):
    print(b_1)
    for b_2 in range(b_1, 61):
        #print(b_2)
        for b_3 in range(b_2, 62):
            if b_1 < b_2 and b_2<b_3 and (gcd(b_1, b_2)==1 and gcd(b_2, b_3)==1 and gcd(b_1, b_3)==1):
                delta = compute_Z_0(b_1, b_2, b_3, 2)
                if delta.denominator !=2:
                    print(str((b_1, b_2, b_3)) + " produces Delta_0: " + str(delta))
                else:
                    counter +=1
print(counter)
print("Done.")

2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
10287
Done.
